In [66]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline , make_pipeline 
from sklearn.preprocessing import OneHotEncoder , MinMaxScaler
from sklearn.feature_selection import SelectKBest , chi2
from sklearn.tree import DecisionTreeClassifier

In [67]:
df = pd.read_csv("D:\\train (5).csv")

In [68]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
693,694,0,3,"Saad, Mr. Khalil",male,25.0,0,0,2672,7.2250,NaN,C
101,102,0,3,"Petroff, Mr. Pastcho (""Pentcho"")",male,NaN,0,0,349215,7.8958,NaN,S
481,482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0000,NaN,S
852,853,0,3,"Boulos, Miss. Nourelain",female,9.0,1,1,2678,15.2458,NaN,C
539,540,1,1,"Frolicher, Miss. Hedwig Margaritha",female,22.0,0,2,13568,49.5000,B39,C


In [69]:
df.drop(columns=['Name','PassengerId','Ticket','Cabin'],inplace=True)

In [70]:
x_train , x_test , y_train , y_test = train_test_split(df.drop(columns=['Survived']),
                                                       df['Survived'],
                                                       test_size=0.2,
                                                       random_state=42)
x_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
606,3,male,30.0,0,0,7.8958,S
148,2,male,36.5,0,2,26.0000,S
453,1,male,49.0,1,0,89.1042,C
674,2,male,NaN,0,0,0.0000,S
287,3,male,22.0,0,0,7.8958,S


In [71]:
y_train.sample(5)

851    0
115    0
347    1
149    0
160    0
Name: Survived, dtype: int64

In [72]:
trf1 = ColumnTransformer ([('impute_age',SimpleImputer(),[2]),
                           ('impute_embarked',SimpleImputer(strategy = 'most_frequent'),[6])
                          ],remainder = 'passthrough')

In [73]:
trf2 = ColumnTransformer([('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
                         ],remainder='passthrough')

In [81]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))])

In [82]:
trf4 = SelectKBest (score_func=chi2 , k=5)

In [83]:
trf5 = DecisionTreeClassifier()

In [84]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [85]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [87]:
pipe.fit(x_train,y_train)

D:\anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x0000022B66E122A0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [88]:
y_pred = pipe.predict(x_test)

In [89]:
from sklearn.metrics import accuracy_score 

In [90]:
accuracy_score(y_test,y_pred)

0.6256983240223464

In [91]:
params = {
    'trf5_max_depth':[1,2,3,4,5,None]
}

In [92]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

ValueError: Invalid parameter 'trf5_max_depth' for estimator Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transfor

In [ ]:
import pickle 
pickle.dump(pipe,o